In [ ]:
# Test for AI module: fdd_utils/ai_helper.py
from fdd_utils.ai_helper import AIHelper
 
# Define system prompt and user prompt
system_prompt = r"""You are a helpful assistant."""
user_prompt = r"""Whehre is HK?"""
 
# Choose the model type ('openai', 'local', 'deepseek')
model_type = "local"
 
# Create an instance of the AIHelper
ai_helper = AIHelper(model_type=model_type, user_prompt=user_prompt, system_prompt=system_prompt)
 
# Get the response from the AI model
response = ai_helper.get_response()
print(response)
 
# AI module test pass
 


In [ ]:
# Databook handling
from fdd_utils.process_databook import extract_data_from_excel
import warnings
 
# Ignore the user warning for data validation
warnings.simplefilter(action='ignore', category=UserWarning)
 
# Usage:
# Eng (Haining 3 entites): 221128.Project TK.Databook.JW.xlsx
# Semi Databook: deli.xlsx
databook_path = "inputs/221128.Project TK.Databook.JW.xlsx"
# Chi (single entity): 240624.联洋-databook.xlsx || 240627.东莞岭南-databook.xlsx
# databook_path = "inputs/240627.东莞岭南-databook.xlsx"
entity_name = 'Haining Wanpu' # Lianyang, Dongguan Lingnan
mode = "BS" # BS, IS, All
 
dfs, workbook_list, result_type, report_language = extract_data_from_excel(databook_path, entity_name, mode)
 
# workbook_list
print('Key list:', workbook_list)
 
# Example databook data for single item
# dfs['Cash']
 
# Excel processing module ok, individual steps inside workings/process_databook.ipynb
# Results -> Pass
# Remark: here key list = workbook tabname
# 联洋, 东莞岭南 IS, BS - pass
 
# one issue: 可能會有subtotal+細項的表達方法, 之後要小心，如果AI有bug要返呢度改
 



In [ ]:
dfs['Cash']
 


In [ ]:
report_language
 


In [ ]:
# Key reconciliation
 
# mapping.json
# "Cash", "AR", "Prepayments", "OR", "Inventory", "IP", "IA", "NCA", "NCL", "Other NCA", "Other CA", "AP", "Advances",
# "Tax payable", "OP", "Capital", "Reserve", "Capital reserve", "R/E", "OI", "OC", "Tax and Surcharges", "GA",
# "Fin Exp",  "Cr Loss", "Other Income", "Non-operating Income", "Non-operating Exp", "Income tax", "LT DTA", "Selling cost"
 
# pattern.json
# "Cash", "AR", "Prepayments", "OR", "IP", "NCA", "Other NCA", "Other CA", "AP", ## 少左: "Inventory"， "IA"， "NCL", "Advances"
# "Tax payable", "OP", "Capital", "Reserve", "OI", "OC", "Tax and Surcharges", "GA", ## 少左: "Capital reserve", "R/E"
# "Fin Exp", "Cr Loss", "Other Income", "Non-operating Income", "Non-operating Exp", "Income tax", "LT DTA" ## 少左: "Capital reserve", "R/E", "Selling cost"
 
# prompts.json
# "Cash", "AR", "Other NCA", "Other CA", "AP" ## 少左: "Prepayments", "OR", "Inventory", "IP", "IA", "NCA", "NCL", "Advances"
# "Reserve", "Capital reserve", "R/E", "OI", "OC", "Tax and Surcharges", "GA", ## 少左: "Tax payable", "OP", "Capital",
# "Fin Exp",  "Cr Loss", "Other Income", "Non-operating Income", "Non-operating Exp", "Income tax", "LT DTA", ## 少左: "Selling cost"
 
# 到時要加返上去 以mappings爲準
# Keys checked identical    
 


In [ ]:
# AI Agent 由呢度開始要+language
 
# Agent 1: 根據acct情況揀pattern+draft Input=language, key(account, pattern) Output=key+AI output
# Agent 2：對數字(+desc)係咪岩 Input=AI output + key(accounts)
# Agent 3: compliance -> 檢查内容係咪make sense
# Agent 4: translate
 
# 盡量+heuristic features 尤其on 容易睇到問題嘅位置
 
# Expand options: 1. industry 2.RAG
 
 


In [ ]:
import pandas as pd
from fdd_utils.content_generation import content_generate
 
agent = 'agent_1'
model_name = 'local'
report_language = 'Eng' # Eng, Chi
 
content_generate(agent, workbook_list, model_name, report_language, dfs)


In [ ]:
# Step 2: Run Agent 2 to verify and refine the output of Agent 1
for mapping_key in mapping_keys:
    result = load_and_verify(agent2, mapping_key)  # Implement a function to load and check results
 
    if result:
        # Implement AI processing for verification and refinement
        system_prompt = f"System prompt for {agent2} verification."
        user_prompt = f"User prompt containing data verification specifics for {mapping_key}."
           
        ai_helper = AIHelper(model_type=model_name, user_prompt=user_prompt, system_prompt=system_prompt)
        verification_response = ai_helper.get_response().strip()
 
        # Final verification step: ensure format compliance and log or output appropriately
        verification_response = verification_response.replace("\n\n", "\n").replace("\n \n", "\n")
        print(f"Verified Output for key {mapping_key}: {verification_response}")
 

In [ ]:
import os, yaml
 
def read_result_by_key(mapping_key, log_dir='fdd_utils'):
    intermediate_file = os.path.join(log_dir, 'intermediate_results.yml')
    if not os.path.exists(intermediate_file):
        print("Intermediate file does not exist.")
        return None
 
    with open(intermediate_file, 'r', encoding='utf-8') as file:
        data = yaml.safe_load(file) or {}
 
    result_data = data.get(mapping_key, None)
    return result_data.get('result') if result_data else None
 
# Example usage to read the result by key
result = read_result_by_key('AR')
print(result)
 
# ['Cash', 'AR', 'Prepayments', 'OR', 'Other current assets', 'Investment properties', 'Tax payable', 'OP', 'AP', 'Share capital']
# AR - Haining issue
# General: 1 d.p., 1000Kshould = M 3. all amount should be CNYxxxK or million
 
# Agent 1 completed with some issues